In [2]:
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from pathlib import Path
from typing import List
from fishsense_api_sdk.models.species_label import SpeciesLabel
from tqdm.asyncio import tqdm_asyncio
from tqdm.notebook import tqdm
from fishsense_api_sdk.models.dive_slate_label import DiveSlateLabel

In [3]:
URL = "http://localhost:8000"
PROJECT_ID = 66

In [4]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [5]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [6]:
async with Client(base_url=URL) as fs:
    dives = await fs.dives.get_canonical()
    dives = [dive for dive in dives if dive.priority == "HIGH"]

len(dives), dives[0]

(7,
 Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=7))

In [7]:
species_labels: List[SpeciesLabel] = await tqdm_asyncio.gather(*[fs.labels.get_species_labels(dive.id) for dive in dives])
species_labels = [label for sublist in species_labels for label in sublist]

len(species_labels), species_labels[0]

100%|██████████| 7/7 [00:01<00:00,  4.89it/s]


(1178,
 SpeciesLabel(id=1024, label_studio_task_id=222252, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/7252dfb0db0b855616e45607f58c1c93', updated_at=datetime.datetime(2025, 10, 20, 20, 42, 14, 832143, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1970.0, laser_y=1340.0, laser_label='Red Laser', content_of_image='Slate, Laser not on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117942, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 49.08221189998697, 'y': 44.44128413294742, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction-changed', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'm4K68yydiZ', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser not on

In [8]:
slate_labels = [label for label in species_labels if label.content_of_image == 'Slate, Laser on slate']

len(slate_labels), slate_labels[0]

(95,
 SpeciesLabel(id=1044, label_studio_task_id=222272, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/2863b55ffedf2bbb1e94f8042fd8ad60', updated_at=datetime.datetime(2025, 10, 21, 1, 8, 15, 395055, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1851.0, laser_y=902.0, laser_label='Red Laser', content_of_image='Slate, Laser on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117986, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 46.10335466840985, 'y': 29.916128498866595, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'xON5aKx1k3', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser on slate']]}, 'origin'

In [9]:
async with Client(URL) as fs:
    slate_images = await tqdm_asyncio.gather(*[fs.images.get(image_id=label.image_id) for label in slate_labels])

len(slate_images), slate_images[0]

100%|██████████| 95/95 [00:02<00:00, 35.45it/s]


(95,
 Image(id=78686, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130190.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 52, tzinfo=TzInfo(0)), checksum='2863b55ffedf2bbb1e94f8042fd8ad60', is_canonical=True, dive_id=279, camera_id=5))

In [10]:
tasks = []

for image in tqdm(slate_images):
    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/{image.checksum}"
        },
        "predictions": [],
        "annotations": []
    })

tasks

  0%|          | 0/95 [00:00<?, ?it/s]

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/2863b55ffedf2bbb1e94f8042fd8ad60'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/ae65a40aae9d2a8af6cc440775b5f64a'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/a97161b74530f64d7b364108ea098d20'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/ff7e76aff9bdd3147a3ffd1942df9f27'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/9008aea58024e6d14d7e6516b7897e5e'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/917db3f3dcf07f1812dfb0b60c8c66e5'},
  'predicti

In [11]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ProjectsImportTasksResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=0.16456055641174316, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=95, prediction_count=0, task_ids=[223832, 223833, 223834, 223835, 223836, 223837, 223838, 223839, 223840, 223841, 223842, 223843, 223844, 223845, 223846, 223847, 223848, 223849, 223850, 223851, 223852, 223853, 223854, 223855, 223856, 223857, 223858, 223859, 223860, 223861, 223862, 223863, 223864, 223865, 223866, 223867, 223868, 223869, 223870, 223871, 223872, 223873, 223874, 223875, 223876, 223877, 223878, 223879, 223880, 223881, 223882, 223883, 223884, 223885, 223886, 223887, 223888, 223889, 223890, 223891, 223892, 223893, 223894, 223895, 223896, 223897, 223898, 223899, 223900, 223901, 223902, 223903, 223904, 223905, 223906, 223907, 223908, 223909, 223910, 223911, 223912, 223913, 223914, 223915, 223916, 223917, 223918, 223919, 223920, 223921, 223922, 223923, 223924, 223925, 223926])

In [13]:
dive_slate_labels = [DiveSlateLabel(id=None, image_id=image.id, label_studio_task_id=task_id, label_studio_project_id=PROJECT_ID, image_url=f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/{image.checksum}", updated_at=None, completed=False, label_studio_json={}, user_id=None) for task_id, image in zip(imported_tasks.task_ids, slate_images)]

len(dive_slate_labels), dive_slate_labels[0]

(95,
 DiveSlateLabel(id=None, label_studio_task_id=223832, label_studio_project_id=66, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/2863b55ffedf2bbb1e94f8042fd8ad60', updated_at=None, completed=False, label_studio_json={}, image_id=78686, user_id=None))

In [14]:
async with Client(base_url=URL) as fs:
    for dive_slate_label in tqdm(dive_slate_labels):
        await fs.labels.put_dive_slate_label(dive_slate_label.image_id, dive_slate_label)

  0%|          | 0/95 [00:00<?, ?it/s]